In [1]:
import pandas as pd
import os

In [3]:
%load_ext autoreload

In [8]:
%aimport ddf_utils.ddf_reader

import ddf_utils.ddf_reader as dr

In [9]:
dr.SEARCH_PATH = '../../../'

In [10]:
p1 = dr.ddf_datapoint('ddf--gapminder--hist_pop', 'total_population')

In [11]:
p1.head()

,area,year,total_population
0,afghanistan,1800,3280000.0
1,afghanistan,1820,3280000.0
2,afghanistan,1870,4207000.0
3,afghanistan,1913,5730000.0
4,afghanistan,1950,8151455.0


In [31]:
area = dr.ddf_entities('ddf--gapminder--hist_pop')['area']

In [33]:
area = area.set_index('area')
m0 = area.to_dict()['name']

In [34]:
p1['name'] = p1.area.map(lambda x: m0[x])

In [35]:
p1.head()

,area,year,total_population,name
0,afghanistan,1800,3280000.0,Afghanistan
1,afghanistan,1820,3280000.0,Afghanistan
2,afghanistan,1870,4207000.0,Afghanistan
3,afghanistan,1913,5730000.0,Afghanistan
4,afghanistan,1950,8151455.0,Afghanistan


In [12]:
p2 = dr.ddf_datapoint('ddf--unpop--wpp_pop', 'population', 'country_code,year,age')

In [13]:
p2.head()

,country_code,year,age,population
0,100,1950,0,162.422
1,100,1950,1,153.456
2,100,1950,2,145.560
3,100,1950,3,138.704
4,100,1950,4,132.858


In [14]:
p2 = p2.groupby(['country_code', 'year'])['population'].sum()

In [17]:
p2 = p2.reset_index()

In [18]:
p2.head()

,country_code,year,population
0,4,1950,7752.118
1,4,1951,7839.426
2,4,1952,7934.798
3,4,1953,8038.312
4,4,1954,8150.037


In [22]:
country_code = dr.ddf_entities('ddf--unpop--wpp_pop')['country_code']

In [23]:
country_code = country_code.set_index(['country_code'])

In [28]:
m = country_code.to_dict()['name']

In [29]:
p2['name'] = p2.country_code.map(lambda x: m[x])

In [30]:
p2.head()

,country_code,year,population,name
0,4,1950,7752.118,Afghanistan
1,4,1951,7839.426,Afghanistan
2,4,1952,7934.798,Afghanistan
3,4,1953,8038.312,Afghanistan
4,4,1954,8150.037,Afghanistan


In [36]:
name_gapminder = p1.name.unique()

In [53]:
name_un = p2[p2.country_code<900].name.unique()

In [40]:
len(set(name_gapminder))

254

In [54]:
len(set(name_gapminder).intersection(set(name_un)))

170

so there are some name difference between gapminder and unpop. Try align them to geo entity domain now.

In [44]:
geo = dr.ddf_entities('ddf--gapminder--geo_entity_domain')['country']

In [45]:
geo.columns

Index(['country', 'gwid', 'name', 'world_6region', 'income_groups',
       'landlocked', 'g77_and_oecd_countries', 'main_religion_2008',
       'gapminder_list', 'alternative_1', 'alternative_2', 'alternative_3',
       'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
       'code', 'number', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6',
       'is--country', 'world_4region', 'latitude', 'longitude'],
      dtype='object')

In [46]:
# mapping for gapminder

map_g = {}

for g in name_gapminder:
    map0 = geo['name'] == g
    map1 = geo['gapminder_list'] == g
    
    map_all = map0 | map1
    filtered = geo[map_all]
    if len(filtered) > 0:
        map_g[g] = filtered['country'].values[0]
    else:
        print('not found: ', g)

In [48]:
p1['geo'] = p1.name.map(lambda x: map_g[x])

In [50]:
p1_ = p1[['geo', 'year', 'total_population']]

In [57]:
# mappings for unpop

map_u = {}

cols = ['name', 'gapminder_list', 'alternative_1', 'alternative_2', 'alternative_3',
       'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
       'code', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6']

for g in name_un:
    maps = []
    
    for c in cols:
        maps.append(geo[c] == g)
    
    map_0 = maps[0]
    
    for mm in maps[1:]:
        map_0 = map_0 | mm
    
    filtered = geo[map_0]
    if len(filtered) > 0:
        map_u[g] = filtered['country'].values[0]
    else:
        print('not found: ', g)

not found:  Other non-specified areas
not found:  Democratic Republic of the Congo
not found:  Iran (Islamic Republic of)
not found:  Republic of Korea
not found:  Lao People's Democratic Republic
not found:  United Republic of Tanzania


In [ ]:
# manually add some geo here

In [63]:
geo[geo.name.str.contains('Tanzania')]

,country,gwid,name,world_6region,income_groups,landlocked,g77_and_oecd_countries,main_religion_2008,gapminder_list,alternative_1,...,arb1,arb2,arb3,arb4,arb5,arb6,is--country,world_4region,latitude,longitude
237,tza,i223,Tanzania,sub_saharan_africa,low_income,coastline,g77,christian,Tanzania,"Tanzania, U. Rep. of",...,NaN,NaN,NaN,NaN,NaN,NaN,True,africa,-6.0,35.0


In [67]:
map_u['Other non-specified areas'] = None
map_u['Democratic Republic of the Congo'] = 'cod'
map_u['Iran (Islamic Republic of)'] = 'irn'
map_u['Republic of Korea'] = 'kor'
map_u["Lao People's Democratic Republic"] = 'lao'
map_u['United Republic of Tanzania'] = 'tza'

In [132]:
## 

p2_ = p2[p2.country_code < 900].copy()

p2_['geo'] = p2_.name.map(lambda x: map_u[x])

In [133]:
p2_ = p2_.dropna(subset=['geo'])

In [134]:
p2_ = p2_[['geo', 'year', 'population']]

In [135]:
p2_.head()

,geo,year,population
0,afg,1950,7752.118
1,afg,1951,7839.426
2,afg,1952,7934.798
3,afg,1953,8038.312
4,afg,1954,8150.037


In [137]:
geo_g = p1_.geo.unique()
geo_u = p2_.geo.unique()

In [138]:
len(set(geo_g).intersection(set(geo_u)))

199

In [139]:
len(set(geo_g))

254

In [140]:
len(set(geo_u))

200

In [ ]:
## so above means only 199 country have interaction of data in both dataset.

In [141]:
p1_ = p1_.set_index(['geo', 'year'])
p2_ = p2_.set_index(['geo', 'year'])

In [86]:
merged = pd.concat([p1_, p2_], axis=1)

In [87]:
merged.population = merged.population * 1000

In [89]:
merged['merged'] = merged.population.fillna(merged.total_population)

In [90]:
merged.to_csv('tmp.csv')

In [ ]:
# make an other indicator with total_population_with_interpolations

In [91]:
p3 = dr.ddf_datapoint('ddf--gapminder--hist_pop', 'total_population_with_interpolations')

In [92]:
p3['name'] = p3.area.map(lambda x: m0[x])

In [93]:
p3['geo'] = p3.name.map(lambda x: map_g[x])

In [94]:
p3_ = p3[['geo', 'year', 'total_population_with_interpolations']]

In [95]:
p3_ = p3_.set_index(['geo', 'year'])

In [125]:
merged_2 = pd.concat([p3_, p2_], axis=1)
merged_2.population = merged_2.population * 1000
merged_2['merged'] = merged_2.population.fillna(merged_2.total_population_with_interpolations)

In [142]:
diffs = merged_2[(merged_2.index.get_level_values(1) >= 1940) & (merged_2.index.get_level_values(1) <= 1960)]

In [143]:
diffs.columns

Index(['total_population_with_interpolations', 'population', 'merged'], dtype='object')

In [144]:
diffs.columns = ['total_population_with_interpolations(GW)', 'populatioin(UN)', 'merged']

In [148]:
diffs[diffs.index.get_level_values(0) == 'irn']

total_population_with_interpolations(GW)  populatioin(UN)  \
geo year                                                              
irn 1940                                15378639.0              NaN   
    1941                                15571000.0              NaN   
    1942                                15765767.0              NaN   
    1943                                15962970.0              NaN   
    1944                                16162640.0              NaN   
    1945                                16364808.0              NaN   
    1946                                16569504.0              NaN   
    1947                                16776760.0              NaN   
    1948                                16986610.0              NaN   
    1949                                17199083.0              NaN   
    1950                                17414215.0       17119263.0   
    1951                                17772538.0       17516578.0   
    1952                                18153564.0       17933356.0   
    1953                                18557412.0       18368896.0   
    1954                                18984036.0       18822605.0   
    1955                                19433222.0       19293998.0   
    1956                                19904579.0       19782699.0   
    1957                                20397551.0       20288440.0   
    1958                                20911449.0       20811062.0   
    1959                                21445515.0       21350524.0   
    1960                                21999103.0       21906905.0   

              merged  
geo year              
irn 1940  15378639.0  
    1941  15571000.0  
    1942  15765767.0  
    1943  15962970.0  
    1944  16162640.0  
    1945  16364808.0  
    1946  16569504.0  
    1947  16776760.0  
    1948  16986610.0  
    1949  17199083.0  
    1950  17119263.0  
    1951  17516578.0  
    1952  17933356.0  
    1953  18368896.0  
    1954  18822605.0  
    1955  19293998.0  
    1956  19782699.0  
    1957  20288440.0  
    1958  20811062.0  
    1959  21350524.0  
    1960  21906905.0

In [149]:
diffs.to_csv('../bridge_diff.csv')

In [153]:
dps = merged_2[['merged']]
dps = dps.reset_index()

In [172]:
dps.columns = ['country', 'year', 'population']

In [173]:
dps.to_csv('../../ddf--datapoints--population--by--country--year.csv', index=False)

# geo entity

In [168]:
geo_u = list(map_u.values())
geo_g = list(map_g.values())

In [159]:
geo_d = list(geo.country)

In [164]:
# geos in UN but not in GW

set(geo_u).difference(set(geo_g))

{'nld_curacao', None}

In [166]:
# geos in GW but not in UN

set(geo_g).difference(set(geo_u))

{'aia',
 'akr_a_dhe',
 'ala',
 'and',
 'ant',
 'asm',
 'bmu',
 'cck',
 'cheslo',
 'cok',
 'cxr',
 'cym',
 'deu_east',
 'deu_west',
 'dma',
 'eri_a_eth',
 'flk',
 'fro',
 'gbg',
 'gbm',
 'gib',
 'grl',
 'hos',
 'jey',
 'kna',
 'korea_union',
 'kos',
 'lie',
 'mco',
 'mhl',
 'mnp',
 'msr',
 'nfk',
 'niu',
 'nru',
 'pcn',
 'plw',
 'scg',
 'scg_ex_kos',
 'shn',
 'sjm',
 'smr',
 'spm',
 'stbar',
 'stmar',
 'tca',
 'tkl',
 'tuv',
 'twn',
 'ussr',
 'vgb',
 'wlf',
 'yem_north',
 'yem_south',
 'yug'}

In [161]:
dps = dps.reset_index()

In [169]:
dps[dps.geo == 'nld_curacao'].head()

,geo,year,population
58622,nld_curacao,1950,100184.0
58623,nld_curacao,1951,104566.0
58624,nld_curacao,1952,108285.0
58625,nld_curacao,1953,111501.0
58626,nld_curacao,1954,114347.0


In [171]:
# for ddf, we just use the geo domain
geo[['country', 'name']].to_csv('../../ddf--entities--country.csv', index=False)

# concept

In [174]:
cdf = dr.ddf_concepts('ddf--gapminder--life_expectancy')['concepts']

In [175]:
cdf

,concept,name,concept_type,indicator_url
0,name,Name,string,NaN
1,time,Time,time,NaN
2,life_expectancy_at_birth_years,Life expectancy at birth (years),measure,https://github.com/open-numbers/ddf--gapminder...
3,geo,Country,entity_domain,NaN
4,indicator_url,Indicator URL,string,NaN


In [176]:
cdf.ix[2] = ['population', 'Population, total', 'measure', 'https://github.com/open-numbers/ddf--gapminder--population']

In [185]:
cdf.ix[3, 'concept'] = 'country'
cdf.ix[1, 'concept'] = 'year'

In [186]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [178]:
from ddf_utils.index import create_index_file

In [187]:
create_index_file('../../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
2,concept,indicator_url,ddf--concepts.csv
3,"country,year",population,ddf--datapoints--population--by--country--year...
4,country,name,ddf--entities--country.csv


In [188]:
!validate-ddf ../../ | jq

[
  {}
]
